This is a simple agent, that scores more that 700. This agent will adapt to each opponent.

**If you find this notebook helpful, please slap the vote button!**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
from sklearn.linear_model import SGDClassifier # Adjust the model

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The following function is designed to get which player won the last turn.

In [ ]:
def winner(p1, p2):
    # This function will return (1, 0) if p1 beat p2,
    # (0, 1) if p2 beat p1 and (0 ,0) for tie.
    if (p1 == 0 and p2 == 1) or (p1 == 1 and p2 == 2) or (p1 == 2 and p2 == 0):
        return (0, 1)
    elif (p1 == 0 and p2 == 2) or (p1 == 1 and p2 == 0) or (p1 == 2 and p2 == 1):
        return (1, 0)
    else:
        return (0, 0)

The next piece of code will build binary features for the last nth plays. To make myself clear, let's say that the last opponent 3 plays was: rock, paper and scissors. So, we have a list of the opponent moves like [0, 1, 2]. When we pass this list to the function, it will build a new list as following [1, 0, 0, 0, 1, 0, 0, 0, 1]. Check out the dictionary map_moves for further comprehension.

In [ ]:
def build_features(opponent_moves):
    # Builds a new set of features where rock (0) will be mapped to (1, 0, 0),
    # paper (1) to (0, 1, 0) and scissors (2) to (0, 0, 1)
    map_moves = {0: (1, 0, 0), 1: (0, 1, 0), 2: (0, 0, 1)}
    features = []
    for move in opponent_moves:
        features.extend(map_moves.get(move))
    return features

The class below is designed to learn the last move online and predict your opponent's next move. The learning process is done using the last 50 opponent moves through logistic regression.

In [ ]:
class MyAgent(object):

    def __init__(self):
        self.cls = SGDClassifier(warm_start=True)

    def train(self, opponent_moves):
        x, y = opponent_moves[0:-1], [opponent_moves[-1]]
        new_features = build_features(x)
        x = np.array(new_features).reshape((1, -1))
        self.cls.partial_fit(x, y, classes=[0, 1, 2])

    def opponent_next_move(self, opponent_moves):
        x = opponent_moves[1:]
        new_features = build_features(x)
        x = np.array(new_features).reshape((1, -1))
        return self.cls.predict(x)[0]

Finally, we have the function of the agent. On the first call to this function, three permanent function variables will be created (the first prediction for the opponent's next move will be rock). One indicating the number of movements of the opponent that will be stored, one with the last nth movements of the opponent (initially all zero) and the last containing the agent object.

In [ ]:
def my_agent(observation, configuration):

    if observation.step > 0:

        # This if statement will ensure that the opponent's 
        # number of moves will not exceed the number of moves
        # that can be stored, erasing the oldest move.
        if len(my_agent.opponent_moves) >= my_agent.len_opponent_moves:
            del my_agent.opponent_moves[0]

        my_agent.opponent_moves.append(observation.lastOpponentAction) # Append the last opponent play
        my_agent.ai.train(my_agent.opponent_moves) # Training the agent using the last turn
        opponent_next_move = my_agent.ai.opponent_next_move(my_agent.opponent_moves) # Predicting the next move
    
        # Once we have the most likely opponet's play
        # we can check which of the options (rock, paper, scissors)
        # will ensure our victory
        for i in (0, 1, 2):
            ans = winner(i, opponent_next_move)
            if ans == (1, 0):
                return i
        return observation.lastOpponentAction
    else:
        my_agent.len_opponent_moves = 51 # Number opponent moves storage
        my_agent.opponent_moves = my_agent.len_opponent_moves * [0] # List of last nth opponent moves
        my_agent.ai = MyAgent() # Agent object for training and preditions
        return 0

A disadvantage here is that the agent is trained from scratch for each opponent. An idea here is to pre-train this agent, to achieve better results!